In [54]:
import pandas as pd
import os
from nltk import sent_tokenize
from tqdm import tqdm
tqdm.pandas()

from transformers_helper import load_tokenizer_and_model, FeatureExtractor
from CustomDataset import encode_for_inference
import finetuning_classification

(source) dvd -> electronics (target)

In [26]:
# dvd -> electronics
result_dir = '/media/dmlab/My Passport/DATA/cross-domain/test_using_post-trained/target=electronics_source=dvd_raw'
filepath = '/media/dmlab/My Passport/DATA/cross-domain/MDSD_masked.json'
finetuned_model_dir = '/media/dmlab/My Passport/DATA/cross-domain/finetune_using_post-trained/source=dvd_raw_post=electronics&dvd'
finetuned_model_dir_no_pt = '/media/dmlab/My Passport/DATA/cross-domain/finetune/source=dvd_raw'
finetuned_model_dir_pt_raw = '/media/dmlab/My Passport/DATA/cross-domain/finetune_using_post-trained_raw/source=dvd_raw_post=dvd&electronics_raw'

post_trained_model_dir = '/media/dmlab/My Passport/DATA/cross-domain/post-train/dvd&books'

* 0: negativve
* 1: positive
* 정답은 positive인데 negative로 잘못 예측한 케이스 250개

In [7]:
pd.read_csv(os.path.join(result_dir, 'confusion_matrix.csv'))

,true,0,1,n_true
0,0,908,65,973
1,1,250,738,988
2,n_pred,1158,803,1961


In [6]:
df = pd.read_csv(os.path.join(result_dir, 'results.csv'))
df.head()

,text,true_label,predicted_label,correct
0,I received my [UNK] 256MB SD [UNK] just as adv...,1,1,True
1,"Works well, especially for anyone who still ha...",1,0,False
2,Not as easy to use as a larger Panasonic I use...,1,1,True
3,It worked much better than expected. The [UNK]...,1,1,True
4,I've had a number of universal [UNK]s over the...,1,1,True


In [10]:
df[(df['true_label']==1)&(df['predicted_label']==0)].values

array([["Works well, especially for anyone who still has their old console systems in use. I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this. It saves tons of sp[UNK]e on my power strip. The design is a little clumsy though, so it's only getting 4/5",
        1, 0, False],
       ['I really don\'t have much to say about this [UNK] holder, not that it\'s just a [UNK] holder. It\'s a nice one. It does it\'s job. It\'s a little too expensive for just a piece of plastic. It\'s strong, sturdy, and it\'s big enough, even for those massive heavy text[UNK]s, like the Calculus ones. Although, I would not recommend putting a dictionary or reference that\'s like 6" thick (even though it still may hold). It\'s got little clamps at the bottom to prevent the page from flipping all over the place, although those tend to fall off when you move them. But that\'s no big deal. Just put them back on. This [UNK] holder is kind of big, and I would not put it on a small [UNK]

문장 단위 classification
* pt UNK
    - doc: negative로 잘못 예측
    - 문장단위로 나누어 예측하면 긍정:부정 = 3:1

In [21]:
tokenizer, model = load_tokenizer_and_model(finetuned_model_dir, mode='classification')

doc = "Works well, especially for anyone who still has their old console systems in use. I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this. It saves tons of sp[UNK]e on my power strip. The design is a little clumsy though, so it's only getting 4/5"
print(finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, doc)))

sents = sent_tokenize(doc)
for sent in sents:
    print('===\n{}\n{}'.format(sent, finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, sent))))

Loading tokenizer & model for /media/dmlab/My Passport/DATA/cross-domain/finetune_using_post-trained/source=dvd_raw_post=electronics&dvd..

0
===
Works well, especially for anyone who still has their old console systems in use.
1
===
I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this.
1
===
It saves tons of sp[UNK]e on my power strip.
1
===
The design is a little clumsy though, so it's only getting 4/5
0


In [24]:
tokenizer, model = load_tokenizer_and_model(finetuned_model_dir_no_pt, mode='classification')

doc = "Works well, especially for anyone who still has their old console systems in use. I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this. It saves tons of sp[UNK]e on my power strip. The design is a little clumsy though, so it's only getting 4/5"
print(finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, doc)))

sents = sent_tokenize(doc)
for sent in sents:
    print('===\n{}\n{}'.format(sent, finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, sent))))

Loading tokenizer & model for /media/dmlab/My Passport/DATA/cross-domain/finetune/source=dvd_raw..

0
===
Works well, especially for anyone who still has their old console systems in use.
1
===
I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this.
1
===
It saves tons of sp[UNK]e on my power strip.
1
===
The design is a little clumsy though, so it's only getting 4/5
0


In [27]:
tokenizer, model = load_tokenizer_and_model(finetuned_model_dir_pt_raw, mode='classification')

doc = "Works well, especially for anyone who still has their old console systems in use. I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this. It saves tons of sp[UNK]e on my power strip. The design is a little clumsy though, so it's only getting 4/5"
print(finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, doc)))

sents = sent_tokenize(doc)
for sent in sents:
    print('===\n{}\n{}'.format(sent, finetuning_classification.inference(model, *encode_for_inference('cpu', tokenizer, sent))))

Loading tokenizer & model for /media/dmlab/My Passport/DATA/cross-domain/finetune_using_post-trained_raw/source=dvd_raw_post=dvd&electronics_raw..

0
===
Works well, especially for anyone who still has their old console systems in use.
1
===
I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this.
1
===
It saves tons of sp[UNK]e on my power strip.
1
===
The design is a little clumsy though, so it's only getting 4/5
0


pipeline
* 영화리뷰로 파인튜닝된 pipeline이 doc을 Positive로 잘 예측하고 있음
* 파인튜닝의 문제가 아니라 post-training의 문제일 수도 있음 
* => 더 많은 데이터를 이용해서 post-training 해보면 doc을 pos로 잘 예측하는지, 전체적인 성능이 향상되는지 확인해보자

In [78]:
from transformers import pipeline
sentiment_pipeline = pipeline('sentiment-analysis')
         
doc = "Works well, especially for anyone who still has their old console systems in use. I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this. It saves tons of sp[UNK]e on my power strip. The design is a little clumsy though, so it's only getting 4/5"
print(sentiment_pipeline(doc))

sents = sent_tokenize(doc)
for sent in sents:
    print('===\n{}\n{}'.format(sent, sentiment_pipeline(sent)))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.6768010854721069}]
===
Works well, especially for anyone who still has their old console systems in use.
[{'label': 'POSITIVE', 'score': 0.9996994733810425}]
===
I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this.
[{'label': 'POSITIVE', 'score': 0.9359228014945984}]
===
It saves tons of sp[UNK]e on my power strip.
[{'label': 'POSITIVE', 'score': 0.9681096076965332}]
===
The design is a little clumsy though, so it's only getting 4/5
[{'label': 'NEGATIVE', 'score': 0.9994996786117554}]


### Data augmentation
* pre-trained model 이용하여 similarity 기반 유사 문장 탐색

In [32]:
raw_df = pd.read_json(filepath)
dvd_df = raw_df[raw_df['domain']=='dvd']
dvd_df.head()

,index,text,label,domain,nouns,keywords,masked_text,masked_text_books&dvd,masked_text_books&electronics,masked_text_books&kitchen,masked_text_dvd&electronics,masked_text_dvd&kitchen,masked_text_electronics&kitchen
1982,1982,"The story of ""Alice in Wonderland"" was controv...",positive,dvd,"[story, alice, wonderland, moment, reverend, d...","[wonderland, rabbit, alice, disney, carroll]","The story of ""[UNK] in [UNK]"" was controversia...","The story of ""[UNK] in [UNK]"" was controversia...",None,None,"The story of ""[UNK] in [UNK]"" was controversia...","The story of ""[UNK] in [UNK]"" was controversia...",None
1983,1983,I covered a similar theme in my international ...,positive,dvd,"[theme, relation, programme, tv, breif, subjec...","[conspiracy, influence, programme, tv]",I covered a similar theme in my international ...,I covered a similar theme in my international ...,None,None,I covered a similar theme in my international ...,I covered a similar theme in my international ...,None
1984,1984,"I saw the scene,where they have Lissa chained ...",positive,dvd,"[scene, pool, table, basement.i, movie, kim, r...","[scene, kim, pool, ron]","I saw the [UNK],where they have Lissa chained ...","I saw the [UNK],where they have Lissa chained ...",None,None,"I saw the [UNK],where they have Lissa chained ...","I saw the [UNK],where they have Lissa chained ...",None
1985,1985,"At 23 years of age, I can truly say that I hav...",positive,dvd,"[year, age, fan, street, dvd, set, street, sch...","[sesame, childhood, gordon]","At 23 years of age, I can truly say that I hav...","At 23 years of age, I can truly say that I hav...",None,None,"At 23 years of age, I can truly say that I hav...","At 23 years of age, I can truly say that I hav...",None
1986,1986,We have all of the series that have been relea...,positive,dvd,"[series, show]",[series],We have all of the [UNK] that have been releas...,We have all of the [UNK] that have been releas...,None,None,We have all of the [UNK] that have been releas...,We have all of the [UNK] that have been releas...,None


소스 문장 단위 임베딩 값 추출

In [34]:
records = []
for _, row in dvd_df.iterrows():
    for sent in sent_tokenize(row['masked_text']):
        records.append((row['index'], sent))
dvd_sent_df = pd.DataFrame(records, columns=['doc_index', 'sentence'])
dvd_sent_df.head()

,doc_index,sentence
0,1982,"The story of ""[UNK] in [UNK]"" was controversia..."
1,1982,Some took the story as horror rather than a ch...
2,1982,Others consider [UNK]'s journey as a trip into...
3,1982,"For better or worse, [UNK] did capture many of..."
4,1982,"However, there are also those who loved the bo..."


In [59]:
feature_extractor = FeatureExtractor(post_trained_model_dir)
dvd_sent_df['pt_unk'] = dvd_sent_df['sentence'].progress_apply(lambda x: feature_extractor.get_feature(x))
dvd_sent_df.head()

Some weights of the model checkpoint at /media/dmlab/My Passport/DATA/cross-domain/post-train/dvd&books were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 38%|███▊      | 7042/18493 [01:44<03:21, 56.82it/s]

ERROR: Under Robert Wise (Director), Herman Schlom (Producer), Eve Greene (Screenwriter), James Gunn (Book Author), Richard Macaulay (Screenwriter), Robert de Grasse (Cinematographer), Constantin Bakaleinikoff (Musical Direction/Supervision), Paul Sawtell (Composer (Music Score), Les Millbrook (Editor), Albert S. D'Agostino (Art Director), Walter E. Keller (Art Director), Sid Rogell (Executive Producer), Darrell Silvera (Set Designer), John Sturtevant (Set Designer), Edward Stevenson (Costume Designer), Russell A. Cully (Special Effects) - - - - the [UNK] includes Lawrence Tierney (Sam Wild), Claire Trevor (Helen Trent), Walter Slezak (Arnold Amett), Phillip Terry (Fred Grover), Audrey Long (Georgia Staples), Elisha Cook, Jr. (Marty Waterman), Isabel Jewell (Laury Palmer), Esther Howard (Mrs. Kraft), Kathryn Card (Grace), Tony Barrett (Danny), Grandon Rhodes (Inspector Wilson), Jason Robards, Sr. (Conductor), Netta Packer (Mrs Perth), Sammy Shack (Crap Dealer), Philip Warren (Chauffeur

 58%|█████▊    | 10669/18493 [02:38<02:06, 61.74it/s]

ERROR: And there is also the horrible music that doesn't fit the mood at all whenever they play at the scene.
RETURN NONE


100%|██████████| 18493/18493 [04:31<00:00, 68.24it/s]


,doc_index,sentence,pt_unk
0,1982,"The story of ""[UNK] in [UNK]"" was controversia...","[-0.13097108051399975, 0.3038369077497491, -0...."
1,1982,Some took the story as horror rather than a ch...,"[-0.042252901675445695, -0.03631691661264215, ..."
2,1982,Others consider [UNK]'s journey as a trip into...,"[-0.1336366706818808, 0.07151144935050979, -0...."
3,1982,"For better or worse, [UNK] did capture many of...","[0.15867450888104298, 0.36486470403478427, -0...."
4,1982,"However, there are also those who loved the bo...","[0.22795244525460637, 0.033146052676088666, 0...."


In [64]:
dvd_sent_df = dvd_sent_df[dvd_sent_df['pt_unk'].apply(lambda x: x is not None)]

Similar sentence
* 효과 없음

In [75]:
dvd_sent_df['doc_label'] = dvd_sent_df['doc_index'].progress_apply(lambda x: dvd_df[dvd_df['index']==x].iloc[0]['label'])

100%|██████████| 18491/18491 [00:10<00:00, 1684.31it/s]
/home/dmlab/anaconda3/envs/torchtext/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
for target_sent in sents:
    doc_vector = feature_extractor.get_feature(target_sent)
    dvd_sent_df['cos'] = dvd_sent_df['pt_unk'].progress_apply(lambda x: cos_sim(doc_vector, x))
    print(target_sent)
    print(dvd_sent_df.sort_values(by=['cos'], ascending=False, axis=0)[:10][['doc_label','sentence']].values[0])

100%|██████████| 18491/18491 [00:00<00:00, 55538.68it/s]
/home/dmlab/anaconda3/envs/torchtext/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Works well, especially for anyone who still has their old console systems in use.
['positive' 'Much better than most things available for teens these days.']


100%|██████████| 18491/18491 [00:00<00:00, 63139.27it/s]


I have my NES and [UNK], both of which have huge [UNK] adapters, plugged into this.
['positive'
 'Note: I have the [UNK] [UNK] drive attached to my Xbox 360 and it works brilliantly !']


100%|██████████| 18491/18491 [00:00<00:00, 63881.93it/s]


It saves tons of sp[UNK]e on my power strip.
['negative'
 '"Rip It Off" and [UNK]s of its ilk give independent [UNK]making a black eye.']


100%|██████████| 18491/18491 [00:00<00:00, 60160.69it/s]

The design is a little clumsy though, so it's only getting 4/5
['negative'
 "-It's also quite obvious who the main [UNK] was, the [UNK] tries a bit of a swerve, but it doesn't really work."]
